In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import pandas
import numpy as np


quadraticInfo = {
    'quadratic': {
        'label': 'Наивное решение',
        'color': 'C0'
    }
}

indexInfo = {
    'bRTreePoint': {
        'label': 'Алгоритм массовой вставки (индексируются события)',
        'color': 'C1'
    },
    'bRTreeRule': {
        'label': 'Алгоритм массовой вставки (индексируются правила)',
        'color': 'C2'
    },
    'xTreePoint': {
        'label': 'X-дерево (индексируются события)',
        'color': 'C3'
    },
    'xTreeRule': {
        'label': 'X-дерево (индексируются правила)',
        'color': 'C4'
    }
}

In [ ]:
def plotScript(resultDir, script):
    info = {**quadraticInfo, **indexInfo}
    
    grouped = {k: pandas.read_csv(f'results/{resultDir}/{script}/{k}.tsv', sep='\t').groupby('dimensions') for k in info}

    j = 1
    f = plt.figure(figsize=(14,8))

    for (i, group) in next(iter(grouped.values())):
        ax = f.add_subplot(2, 3, j)
        ax.grid(True)
        for k in grouped:
            ax.plot(grouped[k].get_group(i)['rules'], grouped[k].get_group(i)['timeNormUs'] / 1000, label=info[k]['label'], color=info[k]['color'])
        ax.margins(0)
        plt.title(f'Размерность пространства: {i}')
        plt.xlabel('Число правил')
        plt.ylabel('Время обработки одного события, мс')
        j += 1

    handles, labels = ax.get_legend_handles_labels()
    f.legend(handles, labels, loc='lower center', fancybox=True, shadow=True, ncol=1)
    f.suptitle(f'Время обработки одного события. Сценарий {script}', size='x-large')
    plt.subplots_adjust(top=0.90, bottom=0.22, left=0.10, right=0.95, hspace=0.3, wspace=0.5)
    plt.show()

In [ ]:
def plotSpeedUp(resultDir, script):
    quadratic = pandas.read_csv(f'results/{resultDir}/{script}/quadratic.tsv', sep='\t').groupby('dimensions')
    
    grouped = {
        k: pandas.read_csv(f'results/{resultDir}/{script}/{k}.tsv', sep='\t').groupby('dimensions') 
        for k in indexInfo
    }
    
    j = 1
    f = plt.figure(figsize=(14,8))
    for (i, group) in quadratic:
        ax = f.add_subplot(2, 3, j)
        ax.grid(True)
        for k in grouped:
            ax.plot(
                grouped[k].get_group(i)['rules'], 
                quadratic.get_group(i)['timeNormUs'] / grouped[k].get_group(i)['timeNormUs'], 
                label=indexInfo[k]['label'], 
                color=indexInfo[k]['color']
            )
        ax.margins(0)
        plt.title(f'Размерность пространства: {i}')
        plt.xlabel('Число правил')
        plt.ylabel('Ускорение')
        j += 1

    handles, labels = ax.get_legend_handles_labels()
    f.legend(handles, labels, loc='lower center', fancybox=True, shadow=True, ncol=1)
    f.suptitle(f'Ускорение. Сценарий {script}', size='x-large')
    plt.subplots_adjust(top=0.90, bottom=0.19, left=0.10, right=0.95, hspace=0.3, wspace=0.5)
    plt.show()

In [ ]:
resultDir = 'reference'
scripts = ['uniform', 'uniformLimited', 'uniformDiscrete', 'gaussian']

In [ ]:
for script in scripts:
    plotScript(resultDir, script)

In [ ]:
for script in scripts:
    plotSpeedUp(resultDir, script)

In [ ]:
def plotPoint(resultDir):
    pointScript = pandas.read_csv(f'results/{resultDir}/pointScript/pointScript.tsv', sep='\t')  
    groupped = pointScript.groupby('rules')
    chunkSizes = [group for (i, group) in groupped['chunkSize']][0].to_numpy()
    ruleSizes = [i for (i, group) in groupped]
    array = np.array([group.to_numpy() / group.max() for (i, group) in groupped['timeNormUs']]) 
    
    ax = plt.gca()
    fig = plt.gcf()

    pcm = ax.pcolor(array, 
                    norm=colors.LogNorm(vmin=array.min(), vmax=array.max()), 
                    cmap='RdYlGn_r'
                   )
    fig.colorbar(pcm, ax=ax, extend='max')
    
    ax.set_yticks(np.arange(len(ruleSizes)))
    ax.set_xticks(np.arange(len(chunkSizes)))
    ax.set_yticklabels(ruleSizes, ha='right', rotation_mode='anchor')
    ax.set_xticklabels(chunkSizes, rotation='vertical', ha='right', rotation_mode='anchor')
    plt.xlabel('Размер группы')
    plt.ylabel('Число правил')
        
    plt.show()
    
    
plotPoint(resultDir)

In [ ]:
def plotComposite(resultDir, script):
    groupIdxs = ['0','1']

    def preparePoints(file):
        source = pandas.read_csv(f'results/{resultDir}/{script}/points/{file}.csv', sep=',')
        return pandas.DataFrame(
            {
                'minutes': (pandas.to_datetime(source['Time']) - pandas.to_datetime(source['Time'][0])).map(lambda x: x.seconds / 60),
                'pps': source[groupIdxs].mean(axis=1)
            }
        )

    def prepareMatches(file):
            source = pandas.read_csv(f'results/{resultDir}/{script}/matches/{file}.csv', sep=',')
            return pandas.DataFrame(
                {
                    'minutes': (pandas.to_datetime(source['Time']) - pandas.to_datetime(source['Time'][0])).map(lambda x: x.seconds / 60),
                    'mps': source['Value']
                }
            )

    f = plt.figure(figsize=(14,4))

    quadraticPoints = preparePoints('quadratic')
    indexPoints = preparePoints('index')

    ax = f.add_subplot(1, 2, 1)
    plt.title('Пропускная способность (события)')
    ax.grid(True)
    ax.plot(quadraticPoints['minutes'], quadraticPoints['pps'], label='Наивное решение')
    ax.plot(indexPoints['minutes'], indexPoints['pps'], label='Индекс')
    meanPPS = indexPoints['pps'].mean()
    ax.hlines(meanPPS, index['minutes'].min(), index['minutes'].max(), label=f'Индекс. Среднее ({int(meanPPS)})')
    ax.legend(loc='lower center', fancybox=True, shadow=True, ncol=1, bbox_to_anchor=(0.5, -0.42))
    plt.xlabel('Время, мин')
    plt.ylabel('Прорускная способность, 1/с')

    quadraticMatches = prepareMatches('quadratic')
    indexMatches = prepareMatches('index')

    ax = f.add_subplot(1, 2, 2)
    plt.title('Пропускная способность (совпадения)')
    ax.grid(True)
    ax.plot(quadraticMatches['minutes'], quadraticMatches['mps'], label='Наивное решение')
    ax.plot(indexMatches['minutes'], indexMatches['mps'], label='Индекс')
    ax.legend(loc='lower center', fancybox=True, shadow=True, ncol=1, bbox_to_anchor=(0.5, -0.35))
    plt.xlabel('Время, мин')
    plt.ylabel('Прорускная способность, 1/с')
    
    f.suptitle(f'Сценарий {script}', size='x-large')

In [ ]:
for script in scripts:
    plotComposite('composite', script)